## Evaluating the systems
Tomamos los usuarios que tengan x+y ratings de peliculas.
Removemos las x peliculas de sus ratings, y pedimos las recomendaciones para las y peliculas.\
Del total de recomendaciones nos quedamos con el TOP z, ordenando por aparicion, y el promedio del cosine_similarity.\
Calculamos recall y precision, variamos x,y,z

In [30]:
import pandas as pd
import numpy as np

In [31]:
BASE_PATH = "dataset"
movies = pd.read_csv(f"{BASE_PATH}/movies_metadata.csv", low_memory=False)

In [32]:
real_shit_indices = pd.Series(movies.index, index=movies['id']).drop_duplicates()

In [34]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
movies['overview'] = movies['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(movies['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(45466, 75827)

In [35]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [82]:
def get_recommendations_and_similarities(data, movie_id, cosine_sim):
    # Get the movie index from dataframe
    idx = real_shit_indices[movie_id]
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]
    
    rows = []
    
    for index, similarity in sim_scores:
        title = data.iloc[index]['title']
        tmdb_id = data.iloc[index]['id']
        # TODO ¿ tener movieId o Index (id fila) ?
        row = {'index':index, 'title':title, 'similarity':similarity, 'tmdbId': tmdb_id}
        rows.append(row)

    # Return the top 10 most similar movies
    return pd.DataFrame.from_records(rows)

In [37]:
user_ratings = pd.read_csv(f"{BASE_PATH}/ratings.csv", dtype={'userId': int, 'movieId': str, 'rating': float,'timestamp': int})

In [38]:
user_ratings.head(2)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435


In [39]:
id_links = pd.read_csv(f"{BASE_PATH}/links.csv", dtype={'movieId': str, 'imdbId': str, 'tmdbId': str})

In [40]:
id_links.head(2)

,movieId,imdbId,tmdbId
0,1,0114709,862
1,2,0113497,8844


In [41]:
user_ratings = pd.merge(user_ratings, id_links, left_on='movieId', right_on='movieId', how='left')

In [42]:
user_ratings.head(2)

,userId,movieId,rating,timestamp,imdbId,tmdbId
0,1,110,1.0,1425941529,0112573,197
1,1,147,4.5,1425942435,0112461,10474


In [107]:
def get_user_recommendations(user, user_ratings_train=user_ratings):
    # WARNING!
    #
    # La matriz de ratings no usa el mismo ID que la matriz movies_metadata
    # En el archivo links se establece una relación entre el movieID de ratings y los IDs de TMBD e IMBD (que el primero parece ser el de movies_metadata)
    print(user)
    movies_and_ratings = user_ratings_train[user_ratings_train['userId'] == user][['tmdbId','rating']]
    out = pd.DataFrame(columns=['index', 'title', 'similarity', "tmdbId"])
    
    for _, info in movies_and_ratings.iterrows():
        # real shit indices se indexa con string, y estos son numeros i.e. '123', y movieId es un float, tonse '123.0' pincha
        movieID = str(int(info.loc['tmdbId']))
        rating = info.loc['rating']
        # ID = id_links[id_links["movieId"] == movieID]["tmdbId"].iloc[0]
        recommendations = get_recommendations_and_similarities(movies, movieID, cosine_sim)
        # Pesa la similaridad * rating, y la normaliza ( /5.0)
        recommendations['similarity'] = recommendations['similarity'] * rating / 5.0
        out = pd.concat([out,recommendations])
        #out = out.append(recommendations, ignore_index=True)
    
    out = out.groupby(['index','title', 'tmdbId'])
    #out['aux'] = None
    #out = out.agg({'similarity':'mean', 'aux':'size'}).rename(columns={'similarity':'mean_similarity','aux':'count'}).reset_index()
    out = out.agg({'similarity':'mean'}).rename(columns={'similarity':'mean_similarity'}).reset_index()
    
    #out.sort_values(by='count', ascending=False, inplace=True)
    out.sort_values(by='mean_similarity', ascending=False, inplace=True)
    to_remove = pd.merge(movies_and_ratings, real_shit_indices.to_frame(), left_on='tmdbId', right_on='id', how='left')
    # Dado que le mergeamos la serie, queda la columna referenciable con el int 0 que son la lista de index del dataframe de movies
    to_remove = to_remove[0].to_list()
    
    out = out[~out['index'].isin(to_remove)][0:10]
    
    
    return out

In [121]:
user_ratings[user_ratings["tmdbId"] == "253768"] # hay varios

,userId,movieId,rating,timestamp,imdbId,tmdbId
538,11,52281,2.5,1231678441,0462322,253768
19672,231,52281,2.0,1195505085,0462322,253768
24144,289,52281,4.0,1281202301,0462322,253768
27397,331,52281,5.0,1490208716,0462322,253768
36141,392,52281,4.5,1197031688,0462322,253768
...,...,...,...,...,...,...
25999668,270634,52281,0.5,1483939994,0462322,253768
26018040,270849,52281,2.5,1441638581,0462322,253768
26018920,270857,52281,4.0,1216899364,0462322,253768
26019366,270864,52281,4.5,1232775799,0462322,253768


In [116]:
get_user_recommendations(11) # tira error sin filtrar

11


KeyError: '253768'

In [86]:
get_user_recommendations(2)

,index,title,tmdbId,mean_similarity
10,1154,The Empire Strikes Back,1891,0.319861
16,2280,Star Trek: Insurrection,200,0.233709
145,30557,Star Trek Beyond,188927,0.212372
19,2659,Better Than Chocolate,18212,0.210412
144,30434,The Star Wars Holiday Special,74849,0.192190
43,6793,Father of the Bride,11846,0.188938
129,26555,Star Wars: The Force Awakens,140607,0.188669
4,324,Star Trek: Generations,193,0.183683
95,18231,Getting to Know You,125506,0.179414
118,23187,The Amazing Spider-Man 2,102382,0.178032


In [99]:
user_ratings_small = user_ratings[:len(user_ratings) // 2**15]
len(user_ratings_small)

794

In [100]:
# 1. Sacar los que tienen menos de 15
user_rating_count = user_ratings_small.groupby(["userId"]).count()
users_to_remove = user_rating_count[user_rating_count["movieId"] < 15].reset_index()["userId"]
filtered_user_ratings = user_ratings_small[~user_ratings_small["userId"].isin(users_to_remove)]

In [101]:
# 2. Sacar 10 de cada uno para test
user_ids = filtered_user_ratings["userId"].unique()
train = filtered_user_ratings
test = pd.DataFrame(columns=filtered_user_ratings.columns)

for user_id in user_ids:
    movies_of_user = filtered_user_ratings[filtered_user_ratings["userId"] == user_id].sample(n=10)
    train.drop(movies_of_user.index, inplace=True)
    test = pd.concat([test, movies_of_user])

/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [102]:
def split_into_chunks(elems: list, chunks: int):
    chunk_size = len(elems)//chunks
    rem = len(elems)%chunks
    chunks_split = [ elems[chunk_size*i:chunk_size*(i+1)] for i in range(0, chunks)]

    # Agregamos el resto al último
    chunks_split[chunks-1].extend(user_ids[len(user_ids) - rem:])

    return chunks_split

In [108]:
%%time
from multiprocess import Process, Manager
from typing import List
# https://stackoverflow.com/questions/10415028/how-can-i-recover-the-return-value-of-a-function-passed-to-multiprocessing-proce

def predict(i: int, return_dict, users: List[int]):
    actual = []
    predicted = []
    print(f"[{i}]: start")
    for user_id in users:
        predicted_movies = get_user_recommendations(user_id, train)["tmdbId"]
        actual_movies = test[test["userId"] == user_id]["tmdbId"]
        
        predicted.append(list(predicted_movies))
        actual.append(list(actual_movies))
        
    return_dict[i] = {
        "pred": predicted,
        "actual": actual,
    }

    print(f"[{i}]: finish")

users_split = split_into_chunks(list(train["userId"].unique()), 2)

procs = []
manager = Manager()
return_dict = manager.dict()
for i, chunk in enumerate(users_split):
    p = Process(target=predict, args=(i, return_dict, chunk))
    p.start()
    procs.append(p)

for p in procs:
    p.join()

predicted = []
actual = []    

for return_value in return_dict.values():
    predicted.extend(return_value["pred"])
    actual.extend(return_value["actual"])
print("Finished!")

[0]: start
1[1]: start

7
2
4
8
5
[0]: finish
9
11


Process Process-33:
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 103, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 135, in pandas._libs.index.IndexEngine._get_loc_duplicates
  File "pandas/_libs/index.pyx", line 143, in pandas._libs.index.IndexEngine._maybe_get_bool_indexer
  File "pandas/_libs/index.pyx", line 161, in pandas._libs.index.IndexEngine._unpack_bool_indexer
KeyError: '253768'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/homebrew/anaconda3/lib/python3.9/site-packages/multiprocess/p

Finished!
CPU times: user 22.6 ms, sys: 52.8 ms, total: 75.5 ms
Wall time: 7.74 s


TODO: se está rompiendo porque hay ratings para peliculas que no aparecen en movies. Filtrar user ratings para que no los tenga.
Si sacamos muchas muchas, hay algo raro, si son pocas, debe ser un error del dataset.